In [2]:
import sqlite3
from sqlite3 import Error

In [16]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

def select_all_tasks(conn, query):
    """
    Executes the provided query.

    :param conn: The Connection object.
    :param query: The SQL query to execute.
    :return: None
    """
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        print(row)

def main():
    database = "sqlite_db_pythonsqlite.db"
    conn = create_connection(database)
    with conn:
        print("Q1. Show all of the facilities that charge a fee to members")
        query = """
            SELECT COUNT(*)
            FROM `Facilities`
            WHERE `membercost` = 0;
         """
        select_all_tasks(conn, query)
        
        print("\nQ2.Number of facilites that do not charge members")
        query = """
            SELECT COUNT(*)
            FROM `Facilities`
            WHERE `membercost` = 0;
        """
        select_all_tasks(conn, query)
        
        print("\nQ3. List of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost")
        query = """
            SELECT `facid`, `name`, `membercost`,`monthlymaintenance`
            FROM `Facilities`
            WHERE `membercost` > 0 AND `membercost` < .2 * `monthlymaintenance`;
        """
        select_all_tasks(conn, query)
        
        print("\nQ4. Retrieve the details of facilities with ID 1 and 5")
        query = """
            SELECT `facid`, `name`, `membercost` FROM `Facilities` WHERE `facid` IN (1,5);
        """
       
        print("\nQ5.  Produce a list of facilities, with each labelled as 'cheap' or 'expensive'")
        query = """
            SELECT `name`, `monthlymaintenance`,
            CASE WHEN `monthlymaintenance` > 100 THEN 'expensive' ELSE 'cheap' END AS `category`
            FROM `Facilities`
            ORDER BY `monthlymaintenance` DESC;
        """
        select_all_tasks(conn, query)
        
        print("\nQ6. Get the first and last name of the last member who signed up.")
        query = """
            SELECT `surname`, `firstname` 
            FROM `Members` 
            WHERE `joindate` = (SELECT MAX(`joindate`) 
            FROM `Members`);
        """
        select_all_tasks(conn, query)
        
        
        print("\nQ7. Produce a list of all members who have used a tennis court.")
        query = """
            SELECT DISTINCT m.`firstname` || ' ' || m.`surname` AS member_name, f.`name` AS `court_name`
            FROM `Bookings` b
            INNER JOIN `Facilities` f ON b.`facid` = f.`facid`
            INNER JOIN `Members` m ON b.`memid` = m.`memid`
            WHERE f.`name` LIKE '%tennis%' AND f.`name` NOT LIKE '%table%';
        """
        select_all_tasks(conn, query)
        
        print("\nQ8. Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30.")
        query = """
        SELECT f.`name` AS facility_name,
           m.`surname` || ', ' || m.`firstname` AS member_name,
        CASE
           WHEN b.`memid` = 0 THEN `slots` * f.`guestcost`
           ELSE `slots` * f.`membercost`
           END AS cost
        FROM Bookings b
        INNER JOIN Facilities f ON b.`facid` = f.`facid`
        LEFT JOIN Members m ON b.`memid` = m.`memid`
        WHERE b.`starttime` >= '2012-09-14'
        AND b.`starttime` < '2012-09-15'
          AND 
          CASE WHEN b.`memid` = 0 THEN `slots` * f.`guestcost`
          ELSE `slots` * f.`membercost`
              END > 30
        ORDER BY `cost` DESC;
        """
        select_all_tasks(conn, query)
        
        print("\nQ9. This time, produce the same result as in Q8, but using a subquery.")
        query = """
        SELECT *
        FROM (
            SELECT f.`name` AS facility_name,
                   m.`surname` || ', ' || m.`firstname` AS member_name,
                   CASE WHEN b.`memid` = 0 THEN f.`guestcost` * b.`slots`
                    ELSE f.`membercost` * b.`slots`
                   END AS cost
            FROM Bookings b
            INNER JOIN `Facilities` f ON b.`facid` = f.`facid`
            LEFT JOIN `Members` m ON b.`memid` = m.`memid`
            WHERE b.`starttime` >= '2012-09-14'
            AND b.`starttime` < '2012-09-15') AS bookings
        WHERE `cost` > 30
        ORDER BY `cost` DESC;
        """
        select_all_tasks(conn, query)
        
        print("\nQ10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember: different cost for guests and members!")
        query = """
        SELECT facs.`name`, SUM(
            CASE WHEN b.`memid` = 0 THEN `slots` * facs.`guestcost` ELSE `slots` * facs.`membercost` END
        ) AS revenue
        FROM `Bookings` b
        INNER JOIN `Facilities` facs ON b.`facid` = facs.`facid`
        GROUP BY facs.`name`
        HAVING SUM(
            CASE WHEN b.`memid` = 0 THEN `slots` * facs.`guestcost` ELSE `slots` * facs.`membercost` END
        ) < 1000
        ORDER BY revenue; 
        """
        select_all_tasks(conn, query)
              
        print("\nQ11: Produce a report of members and who recommended them in alphabetic surname,firstname order")
        query = """
        SELECT `m1`.`surname` AS `member_surname`,
       `m1`.`firstname` AS `member_firstname`,
       `m2`.`surname` AS `recommender_surname`,
       `m2`.`firstname` AS `recommender_firstname`
        FROM `Members` `m1`
        LEFT JOIN `Members` `m2` ON `m1`.`recommendedby` = `m2`.`memid`
        ORDER BY `member_surname`, `member_firstname`;
        """
        select_all_tasks(conn, query)
        
                      
        print("\nQ12: Find the facilities with their usage by member, but not guests")
        query = """
        SELECT `f`.`name` AS `facility_name`, COUNT(`b`.`bookid`) AS `member_usage`
        FROM `Facilities` `f`
        INNER JOIN `Bookings` `b` ON `f`.`facid` = `b`.`facid`
        WHERE `b`.`memid` != 0
        GROUP BY `f`.`name`
        ORDER BY `member_usage` DESC;
        """
        select_all_tasks(conn, query)
              
        print("\nQ13: Find the facilities usage by month, but not guests")
        query = """
        SELECT substr(datetime(b.`starttime`), 6, 2) AS `month`, `f`.`name` AS `facility_name`, COUNT(`b`.`bookid`) AS `member_usage`
        FROM `Facilities` `f`
        INNER JOIN `Bookings` `b` ON `f`.`facid` = `b`.`facid`
        WHERE `b`.`memid` != 0
        GROUP BY `month`, `f`.`name`
        ORDER BY `month`, `member_usage` DESC;
        """
        select_all_tasks(conn, query)
 
if __name__ == '__main__':
    main()

2.6.0
Q1. Show all of the facilities that charge a fee to members
(4,)

Q2.Number of facilites that do not charge members
(4,)

Q3. List of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost
(0, 'Tennis Court 1', 5, 200)
(1, 'Tennis Court 2', 5, 200)
(4, 'Massage Room 1', 9.9, 3000)
(5, 'Massage Room 2', 9.9, 3000)
(6, 'Squash Court', 3.5, 80)

Q4. Retrieve the details of facilities with ID 1 and 5

Q5.  Produce a list of facilities, with each labelled as 'cheap' or 'expensive'
('Massage Room 1', 3000, 'expensive')
('Massage Room 2', 3000, 'expensive')
('Tennis Court 1', 200, 'expensive')
('Tennis Court 2', 200, 'expensive')
('Squash Court', 80, 'cheap')
('Badminton Court', 50, 'cheap')
('Snooker Table', 15, 'cheap')
('Pool Table', 15, 'cheap')
('Table Tennis', 10, 'cheap')

Q6. Get the first and last name of the last member who signed up.
('Smith', 'Darren')

Q7. Produce a list of all members who have used a tennis court.

Thank you!